# Logistic Regressions in R

<iframe width="800" height="450" src="https://www.youtube.com/embed/_QMC94WzcM0" frameborder="0" allowfullscreen></iframe>

## LR as a basic of Machine Learning

**What is machine learning?** 
It’s a Process for building a model

**What is a model then?** 
It’s a function that associates  a response (y) with a set of observable parameters predictors (x1,x2,…): 

$$y= f\left(\vec{x}\right)$$

## Properties of _f(x)_:

- Practical! 
- easy to implement and explain
- It should be single-valued - one output per set of x   vector
- And of course, accurate


## Such as Logistic Regressions!

- One of the categorical models (binomial)

$$logit(p)= ln\left(\frac{p}{1-p}\right) = \alpha + \beta_1X_1 + ... + \beta_kX_k  + \epsilon$$

- Why $logit$ function?
${p}$ is 0 or 1 in our training dataset and is expected to be bounded to [0,1] in prediction where $logit(p)$ could be from $-\infty$ to $\infty$. 

Example:

In [ ]:
df <- mtcars[,c("am", "hp","wt")]

In [ ]:
#when dependant variable (hp) is continuous
library(ggplot2)
ggplot(df,aes(wt,hp))+geom_point()+geom_smooth(method="lm")

In [ ]:
#when dependant variable (am) is binary and running OLS is not appropriate
ggplot(df,aes(wt,am))+geom_point()+geom_smooth(method="lm")

# Implementation in R

<iframe width="800" height="450" src="https://www.youtube.com/embed/AsLWQWwNvok" frameborder="0" allowfullscreen></iframe>

As we can see, the prediction is lower than 0 and it doesn't look like a good fit at all.
So, let's do it in the right way:

In [ ]:
model.fit <- glm(data=df, am ~ wt , family= "binomial")

In [ ]:
summary(model.fit)

$Null\;Deviance = 2(LL(Saturated\;Model)-LL(Null\;Model))$<br>

$Residual\;Deviance = 2(LL(Saturated\;Model)-LL(Fitted\;Model))$<br>


$$Pseudo\;R^2= \frac{Null\;Deviance - Residual\;Deviance}{Null\;Deviance}$$

In [ ]:
print("Sudo R2 is:")
(summary(model.fit)[["null.deviance"]]-summary(model.fit)[["deviance"]])/summary(model.fit)$null.deviance

In [ ]:
head(predict(model.fit))

In [ ]:
head(predict(model.fit, type="response"))

# Performance measurement

<iframe width="800" height="450" src="https://www.youtube.com/embed/SqipRlCK4pI" frameborder="0" allowfullscreen></iframe>

In [ ]:
library(caret)
data(GermanCredit)
df <- GermanCredit[,c("Class","Age","Amount","Duration")]
df$Class <- ifelse(df$Class=="Bad" ,1,0)

In [ ]:
model.fit <- glm(data = df, Class ~ Age + Amount + Duration , family = "binomial")
summary(model.fit)

In [ ]:
ClassHat <- predict(model.fit, type="response")

In [ ]:
library(pROC)
g <- roc(Class ~ ClassHat , data = df)
auc(g)
plot(g, print.thres = "best") 

In [ ]:
fitted.results <- ifelse(ClassHat > 0.50,1,0)
MyTable <- table(fitted.results, df$Class)
caret::confusionMatrix(MyTable)

In [ ]:
cat("Accuracy: " , (35+673)/(27+265+35+673),"\n")
cat("Sensitivity: " , 673/(27+673) ,"\n")
cat("Specifity",35/(35+265),"\n")

What is Kappa?
$$Kappa = \frac{(observed\;accuracy - expected\;accuracy)}{(1 - expected\;accuracy)}$$<br>

In other word, it compares the performance of the model to a way of generating the results by chance. As a rule of thumb, Kappa > 0.75 assumed as an excellent performance.

# Multinomial Regression

<iframe width="800" height="450" src="https://www.youtube.com/embed/lNF62HbOOm8" frameborder="0" allowfullscreen></iframe>

Multinomial Regression is an extension of logistic regression. It is used when the dependent variable is nominal with more than two levels.
In fact, it's a series of logistic regressions in each one class is considered as the event (1) versus one particular class assumed as _pivot_. If we have $K$ classes in our case, we need to run $K-1$ logistic regressions as follows:

$$ln\left(\frac{P(Y_i=1)}{P(Y_i=K)}\right) = \beta_{1}X_i$$<br>

$$ln\left(\frac{P(Y_i=2)}{P(Y_i=K)}\right) = \beta_{2}X_i$$<br>

$$...$$<br>
 
$$ln\left(\frac{P(Y_i=K-1)}{P(Y_i=K)}\right) = \beta_{K-1}X_i$$<br>

Considering that the sum of all probabilities is 1, we can derive that:

$$P(Y_i=K-1) = \frac{e^{\beta_{K-1}X_i}}{\sum_{j=1}^{K}{e^{\beta_j X_i}}}$$

It's also called **softmax** or **normalized exponential** function.

Let's try!

We use `mtcars` dataset in which we have cars with 3,4 and 5 gears. We would like to build a model to predict the gears based on the horse power (`hp`) and weight (`wt`). So, we have 3 classes in this model.

In [ ]:
library(nnet)
df <- mtcars
df$gear <- as.factor(df[["gear"]])
multi.nom <- multinom(gear ~ hp + wt, data= df, maxit=1000, reltol=1e-99, trace=FALSE)

In [ ]:
summary(multi.nom)

In [ ]:
pred <- predict(multi.nom, type="probs")
head(pred)

Now we would like to generate the results as separate logistic regressions, First we need to generate different columns to have 3 separate classes: 

In [ ]:
gear.matrix <- model.matrix(~ gear  - 1, data=df)

In [ ]:
dfXtend <- cbind(df, gear.matrix)
head(dfXtend)

In [ ]:
gear3.glm <- glm(gear3 ~ hp + wt, data=dfXtend, family="binomial")
summary(gear3.glm)

In [ ]:
gear4.glm <- glm(gear4 ~ hp + wt, data=dfXtend, family="binomial")
summary(gear4.glm)

In [ ]:
gear5.glm <- glm(gear5 ~ hp + wt, data=dfXtend, family="binomial")
summary(gear5.glm)

Let's define our softmax function:

In [ ]:
softMax <- function(lr1, lr2, lr3){
  
  p1 <- exp(lr1) / ((exp(lr1) + exp(lr2) + exp(lr3)))
  p2 <- exp(lr2) / ((exp(lr1) + exp(lr2) + exp(lr3)))
  p3 <- exp(lr3) / ((exp(lr1) + exp(lr2) + exp(lr3)))
  
  data.frame(gear3=p1, gear4=p2, gear5=p3)
  
}


In [ ]:
sm.pred <- softMax(predict(gear3.glm, type="link"), predict(gear4.glm, type="link"),  predict(gear5.glm, type="link"))

In [ ]:
head(sm.pred)

In [ ]:
head(pred)

We can test the ranking for all rows as follows:

In [ ]:
max.col(sm.pred)-max.col(pred)

As shown, the ranking is similar for both models.